In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['cleaned_data.csv']


In [2]:
df = pd.read_csv('../input/cleaned_data.csv').iloc[:,1:]
#df = df.drop(["Euc_Dis_To_Hydrology"],axis = 1)
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil_type,Euc_Dis_To_Hydrology,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,1.0,29.0,258.000000,5
1,2590,56,2,212,-6,390,220,235,151,6225,1.0,29.0,211.985849,5
2,2804,139,9,268,65,3180,234,238,135,6121,1.0,12.0,268.121241,2
3,2785,155,18,242,118,3090,238,238,122,6211,1.0,30.0,242.243679,2
4,2595,45,2,153,-1,391,220,234,150,6172,1.0,29.0,152.996732,5


In [3]:
X = df.iloc[:,:-1]
X = X.drop(['Euc_Dis_To_Hydrology'],axis =1)
y = df.iloc[:,-1:]


In [4]:
X.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness,Soil_type
0,2596,51,3,258,0,510,221,232,148,6279,1.0,29.0
1,2590,56,2,212,-6,390,220,235,151,6225,1.0,29.0
2,2804,139,9,268,65,3180,234,238,135,6121,1.0,12.0
3,2785,155,18,242,118,3090,238,238,122,6211,1.0,30.0
4,2595,45,2,153,-1,391,220,234,150,6172,1.0,29.0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Logistic regression**

In [7]:
clf_full = LogisticRegression(multi_class='multinomial',max_iter = 1000,
                         penalty='l2', solver='lbfgs',verbose=1,random_state=21)
clf_full.fit(X_train, np.array(y_train))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=21, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)

In [8]:
clf_full.coef_
 

array([[ 6.23579955e+00, -2.58847223e-01, -3.56633049e-01,
        -2.09951563e-02, -4.23564424e-01, -4.73333595e-01,
        -4.80781829e-01, -5.45388572e-01,  3.92273136e-01,
        -5.48538294e-02, -2.18842981e+00, -2.56287394e-01],
       [ 2.41240544e+00, -2.86701636e-01, -1.03769596e-01,
         4.45004282e-01, -3.31723299e-01, -3.37908526e-01,
        -4.47657596e-01,  1.93408347e-01,  1.50595401e-01,
        -1.05620330e-01, -2.23070080e+00,  1.81040833e-02],
       [-5.70913853e+00,  1.77115437e-01,  2.19079993e-01,
         4.16971391e-01,  3.61281661e-01, -9.48414289e-02,
        -9.07510456e-01,  6.41461861e-01, -1.19560833e+00,
        -4.18832075e-01,  3.14802469e-01, -1.55882375e+00],
       [-8.79110681e+00,  2.07628392e-01,  1.63902945e-01,
        -7.33181155e-01,  6.54412549e-01,  1.86562263e+00,
         8.05880516e-02,  1.60287396e+00, -1.84080852e+00,
         8.23322961e-01,  7.12517815e+00,  5.26575690e-01],
       [ 1.60543831e-01,  1.57269714e-01,  1.9593719

In [9]:
clf_full.intercept_

array([ 1.03947036,  3.18981131,  1.31632947, -5.22622214,  4.01082576,
        2.50202463, -6.83223939])

In [10]:
sparsity = np.mean(clf_full.coef_ == 0) * 100
score = clf_full.score(X_test, y_test)
# print('Best C % .4f' % clf.C_)
print("Sparsity with L2 penalty: %.2f%%" % sparsity)
print("Test score with L2 penalty: %.4f" % score)

Sparsity with L2 penalty: 0.00%
Test score with L2 penalty: 0.7200


In [11]:
import statsmodels.api as st

In [12]:
mdl = st.MNLogit(y_train, X_train)
mdl_fit = mdl.fit(maxiter=1000)


Optimization terminated successfully.
         Current function value: 1.023487
         Iterations 10


In [13]:
print (mdl_fit.summary())

                          MNLogit Regression Results                          
Dep. Variable:             Cover_Type   No. Observations:                15020
Model:                        MNLogit   Df Residuals:                    14948
Method:                           MLE   Df Model:                           66
Date:                Sun, 14 Jul 2019   Pseudo R-squ.:                  0.4740
Time:                        02:35:43   Log-Likelihood:                -15373.
converged:                       True   LL-Null:                       -29228.
Covariance Type:            nonrobust   LLR p-value:                     0.000
Cover_Type=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
x1              -2.2200      0.085    -26.112      0.000      -2.387      -2.053
x2              -0.1817      0.048     -3.793      0.000      -0.276      -0.088
x3               0.6032      0.145      4.16

In [ ]:
kf = KFold(n_splits = 5, shuffle=False)
print('5-fold cv score %f'%cross_val_score(clf_full, X_train, y_train, cv=kf).mean())

In [ ]:
X.head()

In [ ]:
X_sub = np.delete(X_train, (0,1,2,3,), axis=1)
X_sub.shape
X_subtest = np.delete(X_test, (0,2,4,6,8,10), axis=1)

In [ ]:

clf_sub = LogisticRegression(multi_class='multinomial',max_iter = 1000,
                         penalty='l2', solver='lbfgs',verbose=1,random_state=21)
clf_sub.fit(X_sub[:,], np.array(y_train))

In [ ]:
clf_sub.coef_

In [ ]:
sparsity = np.mean(clf_sub.coef_ == 0) * 100
score = clf_sub.score(X_subtest, y_test)
# print('Best C % .4f' % clf.C_)
print("Sparsity with L2 penalty: %.2f%%" % sparsity)
print("Test score with L2 penalty: %.4f" % score)

In [ ]:
kf = KFold(n_splits = 5, shuffle=False)
print('5-fold cv score %f'%cross_val_score(clf_sub, X_sub, y_train, cv=kf).mean())

In [ ]:
mdl = st.MNLogit(y_train, X_train)
mdl_fit = mdl.fit(maxiter=1000)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(X_train)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

In [ ]:
principalDf.head()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
X_train.shape

In [ ]:
clf = LogisticRegression(multi_class='multinomial',max_iter = 1000,
                         penalty='l1', solver='saga',verbose=1,random_state=21)
clf.fit(principalDf, np.array(y_train))

In [ ]:
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(X_test)
principalDf_test = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

In [ ]:
sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(principalDf_test, y_test)
# print('Best C % .4f' % clf.C_)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

In [ ]:
from sklearn.neural_network import MLPClassifier

nh1 = 16
nh2 = 16
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(nh1, nh2), random_state=21).fit(X_train, y_train)
y_pred = mlp.predict(X_test)
print('Neural net accuracy score %s'%metrics.accuracy_score(y_test, y_pred))